In [1]:
from filmModels import *
from dataPreprocess import *
from filmPostProcess import *
import os

c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
label = 'visible_ice'
tab_params = ['proj_x_norm', 'proj_y_norm', 'depth_norm', 'interval_length_norm', 'year_norm', 'month_cyclic_norm']
label_predict = label + '_code'

film=True
chip_size = 64
train_max_epoch = 2
max_iterations = 2
print_train_progress = True
print_test_model = True
hidden_width=32 
hidden_nblocks=4
L2_param = 1e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# path of csv containing tabular data
bh_file_path = 'data_stephen_fix_header.csv'


chips_path = r'C:\Users\mouju\Desktop\film\raster_chips_arcticdem_hds_scaled'

meta_params = {'chip_size': chip_size, 'train_max_epoch': train_max_epoch, 'max_iterations': max_iterations, \
               'print_train_progress': print_train_progress, 'print_test_model':print_test_model, \
              'device':device, 'hidden_width': hidden_width, 'hidden_nblocks':hidden_nblocks, 'L2_param':L2_param, 'film':film}

In [3]:
# list of columns in the dataframe to normalize
list_cols = ['latitude', 'longitude', 'proj_x', 'proj_y', 'depth', 'interval_length', 'time']
df, n_classes = prepare_df(bh_file_path, list_cols, label)

Dataframe has length 2837
Number of unique boreholes is 566
Latitude ranges from 68.36933 to 69.44755
Longitude ranges from -133.82843 to -132.89346
List of columns normalized: ['latitude_norm', 'longitude_norm', 'proj_x_norm', 'proj_y_norm', 'depth_norm', 'interval_length_norm', 'year_norm', 'month_cyclic_norm']
visible_ice: 'None' entries have been replaced by 'No visible ice'
visible_ice column entries has been ordered:
[NaN, 'Pure ice', 'No visible ice', 'High', 'Medium to high', 'Low']
Categories (5, object): ['No visible ice' < 'Low' < 'Medium to high' < 'High' < 'Pure ice']
with corresponding codes in visible_ice_code column:
[-1  4  0  3  2  1]
visible_ice: binary column generated
materials: 'ICE' and 'ice' entries has been standardized into 'Ice'
'material_ice' column generated
materials has been categorized into codes in materials_code
Null entries of visible_ice dropped


In [4]:
df = crop_df_hds(df)

In [5]:
loaded_dataset = filmDataset(chips_path, df, tab_params, label_predict, chip_size)

Dataset contains 5 channels


In [6]:
meta_params['n_channels'] = loaded_dataset.n_channels
meta_params['n_classes'] = n_classes
if n_classes == 1:
    meta_params['loss_fn'] = torch.nn.BCELoss()
else:
    meta_params['loss_fn'] = torch.nn.CrossEntropyLoss()
meta_params['full_dataset'] = loaded_dataset

In [7]:
train_loader, test_loader = split_data(loaded_dataset)

Batch size 20
Data split into training size 573 and testing size 144


In [9]:
results = iterate(meta_params, train_loader, test_loader)

c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch  1: running loss: 1.57695, validation loss: 1.47987
epoch  2: running loss: 1.40572, validation loss: 1.38616
Finished Training
epoch 2 model selected
iteration 1 elapsed time: 6.101025104522705, accuracy : 0.3541666666666667
epoch  1: running loss: 1.45870, validation loss: 1.42108
epoch  2: running loss: 1.38340, validation loss: 1.45319
Finished Training
epoch 1 model selected
iteration 2 elapsed time: 3.4968881607055664, accuracy : 0.3333333333333333


In [10]:
display_results(meta_params, results)

mean


,precision,recall,f1,support
0,0.143678,0.328947,0.200000,38.0
1,0.078947,0.050000,0.061224,30.0
2,0.440241,0.696078,0.486258,51.0
3,0.000000,0.000000,0.000000,6.0
4,0.000000,0.000000,0.000000,19.0


std


,precision,recall,f1,support
0,0.143678,0.328947,0.200000,0.0
1,0.078947,0.050000,0.061224,0.0
2,0.086075,0.303922,0.036819,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0


Accuracy mean: 0.34375, std: 0.010416666666666685


In [11]:
## draw heatmap

In [12]:
# df2 = df.dropna(subset=['visible_ice'])